# Artwork sequence prediction

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os

In [ ]:
#CONFIG_PATH = '/root/work/artwork_sequence/train_test_configuration/multivariate'
CONFIG_PATH = '/root/work/artwork_sequence/train_test_configuration/univariate'
#CONFIG_PATH = '/root/work/artwork_sequence/train_test_configuration/univariate_predict_multiple'

In [ ]:
window_index = 3

museum_sequence_path = {
    'x_train' : os.path.join(CONFIG_PATH, 'X_train.csv'),
    'x_test' : os.path.join(CONFIG_PATH, 'X_test.csv'),
    'x_train_matrix' : os.path.join(CONFIG_PATH, 'X_train_matrix.npy'),
    'x_test_matrix' : os.path.join(CONFIG_PATH, 'X_test_matrix.npy'),
    'weights_folder' : os.path.join(CONFIG_PATH, 'config_'+str(window_index)+'/trained_model_weights')
}
museum_sequence_path

## Load data

In [ ]:
df_x_train = pd.read_csv(museum_sequence_path['x_train'], index_col=0)
df_x_test = pd.read_csv(museum_sequence_path['x_test'], index_col=0)
x_train_matrix = np.load(museum_sequence_path['x_train_matrix'])
x_test_matrix = np.load(museum_sequence_path['x_test_matrix'])
df_x_train.head()

In [ ]:
x_train_matrix.shape

## Reset Tensorflow session

In [ ]:
tf.keras.backend.clear_session()

## Config data to fit with the model input

Because the **Prediction feature model** split the data into training and validation dataset, it is necessary to give all the data in only one block

**Define timeline**

In [ ]:
time = np.arange(x_train_matrix.shape[0] + x_test_matrix.shape[0])
time.shape

**Define configuration to deal with the windowed dataset**

In [ ]:
split_time = x_train_matrix.shape[0]

X = np.concatenate((x_train_matrix, x_test_matrix))

#length of the history
window_size = window_index

#Number of artwork's feature
n_features = X.shape[1]

#Number of feature to take into account
n_influence_features=10

batch_size = 128
shuffle_buffer_size = 300

## Create models

**Define model**

In [ ]:
from Sequence_prediction_factory import Sequence_prediction_multivariate, Sequence_prediction_univariate

In [ ]:
model_univariate = Sequence_prediction_univariate(
    X=X, 
    shuffle_buffer_size=shuffle_buffer_size, 
    split_time=split_time, 
    train_batch_size=batch_size, 
    val_batch_size=batch_size, 
    window_size=window_size)

In [ ]:
model_multivariate = Sequence_prediction_multivariate(
    X=X, 
    shuffle_buffer_size=shuffle_buffer_size, 
    split_time=split_time, 
    train_batch_size=batch_size, 
    val_batch_size=batch_size, 
    window_size=window_size, 
    n_influence_features=n_influence_features)

In [ ]:
model_prediction = model_univariate

In [ ]:
from IPython.display import clear_output
import time

models = []
start_time = time.time()
for i in range(n_features):
    clear_output(wait=True)
    print("---------- Feature %s -------------" % (i))
    
    model_prediction.set_index(i)
    model = model_prediction.get_model()
    model.define_model(conv_filter=16, lstm_filter=32, dense_filter=16, prediction_length=1)
    model.train_model(epochs=20, lr=1e-6)
    models.append(model)
    
    #Save weights
    model.save_weights(museum_sequence_path)
    
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
len(models)